In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
!git clone https://github.com/UrsaTechnology/ControlNet-Projects.git
%cd /ControlNet-Projects/TabCSDI
!pip install -r requirements.txt

fatal: destination path 'TabCSDI' already exists and is not an empty directory.
/content/drive/MyDrive/TabCSDI
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 695.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1

**Train**

Before the training, please upload the dataset csv to the path "TabCSDI"

In [ ]:
main.py --nfold 100

Namespace(config='config.yaml', device='cuda', seed=3047, testmissingratio=0.2, nfold=100, unconditional=0, modelfolder='100_20231029_053837', nsample=50)
{
    "train": {
        "epochs": 10,
        "batch_size": 50,
        "lr": 0.001
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": true,
        "token_emb_dim": 8
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 16,
        "target_strategy": "random",
        "mixed": true,
        "token_emb_dim": 8,
        "test_missing_ratio": 0.2
    }
}
model folder: ./save/100_20231029_062736/
--------Normalized dataset loaded--------
Dataset size:124511 entries
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded------

**Distribution Exploration**

In [ ]:
from src.utils_table import *
from src.main_model_table_ft import *
from data import *
import torch
import datetime
import json
import yaml
import os
import matplotlib.pyplot as plt

indices = [10, 119, 160]
path = './save/100_20231029_053837/model.pth'  # Check the path "save"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open("./config/config.yaml", "r") as f:
    config = yaml.safe_load(f)


config["model"]["is_unconditional"] = False
config["model"]["test_missing_ratio"] = 0.2

model = TabCSDI(config, device).to(device)

model.load_state_dict(torch.load(path))

_, _, test_loader = get_dataloader(seed=3047, batch_size=1, nfold=100)
for i in indices:
  test_loader.dataset.gt_masks[:, i] = False

samples = []

for i in tqdm(range(10)):
  sample, _, _, _, _ = model.evaluate(next(iter(test_loader)), 100)
  sample = sample.permute(0, 1, 3, 2)
  samples.append(sample)

100
100
--------Normalized dataset loaded--------
Dataset size:124511 entries
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
Training dataset size: 123265
Validation dataset size: 0
Testing dataset size: 1246


100%|██████████| 10/10 [12:36<00:00, 75.65s/it]


In [ ]:
with open("./data/transformed_columns.pk", "rb") as f:
    cont_list, num_cate_list = pickle.load(f)
with open("./data/encoder.pk", "rb") as f:
        encoder = pickle.load(f)

decoded_samples = []

for sample in samples:
  temp = torch.zeros(100,224)
  for i in range(100):
    temp[i,:] = model.tokenizer.recover(sample[:,i,:,:], len(cont_list))[0]  # Recover category from embedding
  decoded_samples.append(temp)

In [ ]:
import numpy as np

for i in range(10):
  for ind in [10, 119, 160]:
    plt.hist(np.array(decoded_samples[i][:,ind]),bins=10)
    plt.savefig("{},{}.png".format(i, ind), bbox_inches='tight')
    plt.close()

In [ ]:
for ind in [10, 119, 160]:
  plt.hist([np.array(decoded_samples[i][:,ind]) for i in range(10)],bins=20,stacked=True)
  plt.savefig("{}.png".format(ind), bbox_inches='tight')
  plt.close()

**Single Feature Imputation**

In [ ]:
from src.utils_table import *
from src.main_model_table_ft import *
from data import *
import torch
import datetime
import json
import yaml
import os

indices = [10, 119, 160]
path = './save/100_20231029_053837/model.pth'  # Check the path "save"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open("./config/config.yaml", "r") as f:
    config = yaml.safe_load(f)


config["model"]["is_unconditional"] = False
config["model"]["test_missing_ratio"] = 0.2

model = TabCSDI(config, device).to(device)

model.load_state_dict(torch.load(path))

_, _, test_loader = get_dataloader(seed=3047, batch_size=60, nfold=100, missing_ratio=0)

for i in range(20):
  test_loader.dataset.gt_masks[i, 10] = False
for i in range(20,40):
  test_loader.dataset.gt_masks[i, 119] = False
for i in range(40,60):
  test_loader.dataset.gt_masks[i, 160] = False   # Manual mask

with open("./column_index.pk", "rb") as f:
  column_index = pickle.load(f)

sample, _, _, _, _ = model.evaluate(next(iter(test_loader)), 5)
sample = sample.permute(0, 1, 3, 2)

{'BB textfield': 0, 'Seat tube length': 1, 'Stack': 2, 'Head tube upper extension2': 3, 'Seat angle': 4, 'CS textfield': 5, 'FCD textfield': 6, 'Seat tube extension2': 7, 'Head tube lower extension2': 8, 'Head angle': 9, 'Saddle height': 10, 'ERD rear': 11, 'Wheel width rear': 12, 'Dropout spacing': 13, 'SPOKES composite front': 14, 'SPOKES front': 15, 'Wheel diameter front': 16, 'BSD front': 17, 'Wheel diameter rear': 18, 'SPOKES rear': 19, 'Wheel width front': 20, 'SPOKES composite rear': 21, 'ERD front': 22, 'BSD rear': 23, 'SBLADEW front': 24, 'SBLADEW rear': 25, 'FORK0R': 26, 'FORK0L': 27, 'Saddle P': 28, 'Saddle thickness': 29, 'Saddle angle': 30, 'Saddle J': 31, 'Saddle H': 32, 'Saddle E': 33, 'SADDLETIPtoMIDDLE': 34, 'Saddle length': 35, 'Seatpost setback': 36, 'Seatpost LENGTH': 37, 'Lower stack height': 38, 'Upper stack height': 39, 'Headset spacers': 40, 'Stem angle': 41, 'CLAMPOFFSET': 42, 'StemX': 43, 'StemY': 44, 'StemG': 45, 'Collar height': 46, 'Stem length': 47, 'AEROF

ValueError: ignored

In [ ]:
with open("./data/transformed_columns.pk", "rb") as f:
    cont_list, num_cate_list = pickle.load(f)
with open("./data/encoder.pk", "rb") as f:
    encoder = pickle.load(f)

temp = torch.zeros(300,224)
for i in range(60):
  for j in range(5):
    temp[i*5+j,:] = model.tokenizer.recover(sample[i,j,:,:].unsqueeze(0), len(cont_list))[0]

In [ ]:
data = pd.read_csv("./new_colored_biked_processed.csv", header=0)

masked_saddle = pd.DataFrame(np.repeat(data.values[0:20],5,axis=0))
masked_saddle.columns = data.columns
masked_saddle["Saddle height"] = pd.NA
masked_saddle.to_csv("masked_saddle.csv")

masked_fender = pd.DataFrame(np.repeat(data.values[20:40],5,axis=0))
masked_fender.columns = data.columns
masked_fender["Front fender end angle"] = pd.NA
masked_fender.to_csv("masked_fender.csv")

masked_rack = pd.DataFrame(np.repeat(data.values[40:60],5,axis=0))
masked_rack.columns = data.columns
masked_rack["Display RACK"] = pd.NA
masked_rack.to_csv("masked_rack.csv")

masked_saddle["Saddle height"] = temp[0:100, 10]
masked_saddle.to_csv("generated_saddle.csv")

masked_fender["Front fender end angle"] = temp[100:200, 119]
masked_fender.to_csv("generated_fender.csv")


In [ ]:
masked_rack["Display RACK"] = temp[200:300, 160] - 1
masked_rack.to_csv("generated_rack.csv")

**Multiple Feature Imputation**

In [ ]:
_, _, test_loader = get_dataloader(seed=3047, batch_size=60, nfold=100, missing_ratio=0)

with open("./column_index.pk", "rb") as f:
  column_index = pickle.load(f)

for i in column_index:
  if i not in ["Saddle height", "Display RACK", "Front fender end angle"] and "bottle" not in i:
    test_loader.dataset.gt_masks[:,column_index[i]] = False

sample, _, _, _, _ = model.evaluate(next(iter(test_loader)), 5)
sample = sample.permute(0, 1, 3, 2)

--------Normalized dataset loaded--------
Dataset size:124511 entries
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
--------Normalized dataset loaded--------
Training dataset size: 123265
Validation dataset size: 0
Testing dataset size: 1246


In [ ]:
with open("./data/transformed_columns.pk", "rb") as f:
    cont_list, num_cate_list = pickle.load(f)
with open("./data/encoder.pk", "rb") as f:
        encoder = pickle.load(f)

temp = torch.zeros(100,224)
for i in range(20):
  for j in range(5):
    temp[i*5+j,:] = model.tokenizer.recover(sample[i,j,:,:].unsqueeze(0), len(cont_list))[0]

In [ ]:
data = pd.read_csv("./new_colored_biked_processed.csv", header=0)

masked_saddle = pd.DataFrame(np.repeat(data.values[0:20],5,axis=0))
masked_saddle.columns = data.columns

for i in column_index:
  if i not in ["Saddle height", "Display RACK", "Front fender end angle"] and "bottle" not in i:
    masked_saddle[i] = pd.NA

masked_saddle.to_csv("masked.csv")

for i in column_index:
  if i not in ["Saddle height", "Display RACK", "Front fender end angle"] and "bottle" not in i:
    index = column_index[i]
    if index not in cont_list:
      masked_saddle[i] = temp[:, index] - 1
    else:
      masked_saddle[i] = temp[:, index]

masked_saddle.to_csv("generated.csv")